In [41]:
import numpy as np

pixels = np.array([ (721, 2022), # pixels
                    (1068, 1934),
                    (984, 1804),
                    (1112, 40),
                  ])
physical = np.array([ (179.155, 2), # mm, slit number
                      (133.901, 6),
                      (144.962, 12),
                      (129.033, 92)
                    ])


# Pad the data with ones, so that our transformation can do translations too
n = pixels.shape[0]
pad = lambda x: np.hstack([x, np.ones((x.shape[0], 1))])
unpad = lambda x: x[:,:-1]
X = pad(pixels)
Y = pad(physical)

# Solve the least squares problem X * A = Y
# to find our transformation matrix A
A, res, rank, s = np.linalg.lstsq(X, Y)

transform = lambda x: unpad(np.dot(pad(x), A))

print("Target:")
print(physical)
print("Result:")
print(transform(pixels))
print("Max error:", np.abs(physical - transform(pixels)).max())
A[np.abs(A) < 1e-10] = 0  # set really small values to zero
print('Transformation Matrix')
print(A)

Target:
[[ 179.155    2.   ]
 [ 133.901    6.   ]
 [ 144.962   12.   ]
 [ 129.033   92.   ]]
Result:
[[ 179.16102716    2.01634855]
 [ 133.91694512    6.04325081]
 [ 144.93810778   11.93519282]
 [ 129.03491995   92.00520782]]
Max error: 0.0648071795272
Transformation Matrix
[[ -1.30501561e-01   9.53563140e-05   0.00000000e+00]
 [ -4.54088456e-04  -4.53842457e-02   0.00000000e+00]
 [  2.74170820e+02   9.37145414e+01   1.00000000e+00]]


In [52]:
Ainv, res, rank, s = np.linalg.lstsq(Y, X)

reverse = lambda x: unpad(np.dot(pad(x), Ainv))

print("Target:")
print(pixels)
print("Result:")
print(reverse(physical))
print("Max error:", np.abs(pixels - reverse(physical)).max())
Ainv[np.abs(Ainv) < 1e-10] = 0  # set really small values to zero
print('Reverse Transformation Matrix')
print(Ainv)

Target:
[[ 721 2022]
 [1068 1934]
 [ 984 1804]
 [1112   40]]
Result:
[[  721.04512804  2022.36190124]
 [ 1068.11857426  1934.95089813]
 [  983.82176362  1802.57064561]
 [ 1112.01453407    40.11655501]]
Max error: 1.42935438825
Reverse Transformation Matrix
[[ -7.66267614e+00  -1.60085615e-02   0.00000000e+00]
 [  7.66750274e-02  -2.20338636e+01   0.00000000e+00]
 [  2.09369852e+03   2.06929764e+03   1.00000000e+00]]


In [56]:
reverse(np.array([[179.155, 2]]))

array([[  721.04512804,  2022.36190124]])

In [60]:
transform([[721, 2022]])

AttributeError: 'list' object has no attribute 'shape'

In [57]:
pixels.shape

(4, 2)

In [58]:
physical.shape

(4, 2)